In [1]:
import os
base = "/data/bionets" if "ramses" in os.uname()[1] else "/data_nfs/"

import cv2 
import sys
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import torch as t
from tqdm import tqdm

os.environ["CUDA_VISIBLE_DEVICES"]="1"
sys.path.append("..")
from src import *

2024-08-05 13:39:52.687057: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 13:39:52.700896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 13:39:52.717506: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 13:39:52.722387: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-05 13:39:52.734714: I tensorflow/core/platform/cpu_feature_guar

In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
config_path = "../config.json"
with open(config_path, "r") as f:
    configs = json.load(f)
    
dataset_statistics = configs["dataset_statistics"]
checkpoint_path = configs["downloaded_model_weights"]
pretrained_model_path = configs["pretrained_model_path"]
roi_save_path = configs["PFS_ROIs"]
finetuned_models_path = configs["finetuned_models"]
segmentation_results = configs["segmentation_results"]

with open(os.path.join(configs["dataset_statistics"], "melanoma_means.json"), "r") as f:
    markers = json.load(f).keys()

In [4]:
print(roi_save_path)

/data_nfs/je30bery/melanoma_data/data/ROIs_new/


In [5]:
data = get_data_csv(high_quality_only=True, pfs=True, config_path=config_path)
data = data.reset_index().drop("index", axis=1)

In [6]:
splits = {0: [16],
 1: [29],
 2: [33],
 3: [8, 15, 18, 19, 20],
 4: [25],
 5: [9],
 6: [21],
 7: [5],
 8: [30],
 9: [17],
 10: [3],
 11: [26],
 12: [32],
 13: [4],
 14: [24],
 15: [11, 12],
 16: [0],
 17: [7],
 18: [22, 34],
 19: [28],
 20: [27],
 21: [6]}

In [7]:
model = ResNet18_smoothgrad(indim=len(markers), cam=True, checkpoint_path=checkpoint_path)
model = model.to("cuda:0")
finetuned_models = os.listdir(finetuned_models_path)
gcs = list()
filepaths = list()
for split in tqdm(splits):
    data_subset = data[data["Histo ID"].isin(splits[split])]
    if len(data_subset) == 0:
        continue
    filepaths.append(data_subset["Sample"].values)
    dl = t.utils.data.DataLoader(MelanomaData(markers, pretrain=False, data=data_subset, mode="val", config_path=config_path), batch_size=1, shuffle=False)
    model_path = [m for m in finetuned_models if f"split={split}" in m][0]
    model.load_state_dict(t.load(os.path.join(finetuned_models_path, model_path), map_location="cuda:0"), strict=True)
    model.eval()
    noise_correction = get_smooth_grad(None, model, cuda=True, n_smooth=20, noise_std=0.15)
    corrector = np.mean(noise_correction, axis=-1)
    gc = get_smooth_grad(dl, model, cuda=True, n_smooth=10, noise_std=0.15)
    for i in range(gc.shape[-1]):
        gradcam = gc[:,:,i] 
        gradcam /= np.max(gradcam)
        corrected = gradcam - corrector
        corrected = np.where(corrected > 0, corrected, 0)
        gcs.append(corrected)
filepaths = np.concatenate(filepaths)

100%|██████████████████████████████████████████████████████████████████████████████| 22/22 [20:59<00:00, 57.26s/it]


In [11]:
for filepath, gc in zip(filepaths, gcs):
    segmented = os.path.join(segmentation_results, f'{filepath}_cell.npy')
    with open(segmented, "rb") as openfile:
        seg_file = np.load(openfile)
    img, roi_cells = get_binary(gc, seg_file, output_size=512, cutoff=0.9)
    os.makedirs(roi_save_path, exist_ok=True)
    with open(os.path.join(roi_save_path, filepath + "_idxs.pkl"), "wb") as fp:
        pickle.dump(roi_cells, fp)